In [1]:
import numpy as np
import matplotlib.pyplot as plt
import library.volumes.strmesh as vol
import scipy.io as sio
from sklearn.pipeline import Pipeline
from sklearn.kernel_approximation import RBFSampler
from sklearn import mixture
import open3d as o3d

nolineal=1
xfilename='data/curvature/'
structure='-43-Right-Lateral-Ventricle'#'-2-Left-Cerebral-White-Matter'
surface1='sano-1'+structure
surface2='mv-1'+structure
components=4
montecarlo=200
gamma=0.01

In [2]:
filename=xfilename+surface1+'-'+surface2+'.mat'
info=sio.whosmat(filename)[0]
mat=sio.loadmat(filename)
X=mat[info[0]]

In [3]:
#Dir data
base='data/brain/'
data=surface1+'.mat'
mesh1=vol.StrMesh(filename=base+data)

!Mesh load complete!
Vertices: 2100
Triangles: 4196


In [4]:
base='data/brain/'
data=surface2+'.mat'
mesh2=vol.StrMesh(filename=base+data)

!Mesh load complete!
Vertices: 2433
Triangles: 4862


In [5]:
if nolineal==1:
    #gmm-no lineal
    steps = [('rff', RBFSampler(gamma=gamma,n_components=montecarlo,random_state=48)), 
            ('cluster', mixture.BayesianGaussianMixture(n_components=components, random_state=48))] #clasificador 
    method = Pipeline(steps)
else:
    #gmm lineal
    steps = [ ('cluster', mixture.BayesianGaussianMixture(n_components=components, random_state=48))] #clasificador 
    method = Pipeline(steps) 

In [6]:
method.fit(X)
Z=method.predict(X)

In [ ]:
tam1=mesh1.vertices.shape[0]
tam2=mesh2.vertices.shape[0]
Z1=Z[0:tam1]
Z2=Z[tam1:]
mdic={'Z1':Z1,'Z2':Z2}
savedir='D:\MaestriaLocal\shapecorrespondence\Data\labels'
sio.savemat(savedir+'\label-'+surface1+'-'+surface2+'.mat', mdic)